In [154]:
# Plotting library
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import gridspec

# Numerical Library
import numpy as np
from numpy import genfromtxt
import collections, math, operator

from scipy.special import expit

# Limit printout to 3 decimal places
np.set_printoptions(precision=8,suppress=True)

# Allows for printing inline for jupyter notebook
%matplotlib inline 

In [155]:
## Part 1
DistanceFlagPair = collections.namedtuple('Distance', 'distance flag')

class DataFormat():

    def __init__(self, name):
        self.name = name

    def func_data_set(self):
        src_data = genfromtxt(self.name, delimiter=',')
        flag_data = src_data[:,0]
        feature_data = np.delete(src_data, 0, axis=1)
        return feature_data, flag_data
        
    def func_norm_data(self):
        src_data = genfromtxt(self.name, delimiter=',')
        flag_data, feature_data = src_data[:,0], np.delete(src_data, 0, axis=1)
        feature_max, feature_min = np.amax(feature_data, axis=0), np.amin(feature_data, axis=0)
        feature_range, feature_avg = feature_max - feature_min, np.average(feature_data)
        norm_data = (feature_data - feature_avg) / feature_range
        self.norm_data, self.flag_data = norm_data, flag_data
        return norm_data, flag_data

    def func_leave_out(self, index):
        return self.norm_data[index], self.flag_data[index]

class KNN():

    def __init__(self, norm_data, flag_data):
        self.norm_data, self.flag_data = norm_data, flag_data

    def func_classify(self, test_data, k):
        neigbor_pair = self.func_build_neigbor(test_data)
        flag_sum = np.sum([neigbor.flag for neigbor in neigbor_pair[:k]])
        return -1 if flag_sum < 0 else 1
        
    def func_build_neigbor(self, test_data):
        diff_sqrt = (test_data - self.norm_data) ** 2
        distance_data = np.sqrt(np.sum(diff_sqrt, axis=1))
        neigbor_pair = []
        for i in range(len(distance_data)):
            neigbor = DistanceFlagPair(distance_data[i], self.flag_data[i])
            neigbor_pair.append(neigbor)
        return sorted(neigbor_pair, key=operator.attrgetter('distance'))        

def func_calc_error(data, knn, k = 1):
    temp_data = [knn.func_classify(classifier, k) for classifier in data.norm_data]
    return np.sum(np.abs(temp_data - data.flag_data)) / float(2 * len(data.flag_data))
    
def func_cross_valid_error(data, knn, k = 1):
    error = 0
    for i in range(len(data.norm_data)):
        norm, flag = data.func_leave_out(i)
        flag_sum = knn.func_classify(norm, k)
        flag_error = np.abs(flag - flag_sum) / 2
        error += flag_error
    return float(error) / (len(data.flag_data) + 1) 

In [156]:
## Part 2
class DecisionStump():
    
    def __init__(self, norm_data, flag_data):
        self.norm_data, self.flag_data = norm_data, flag_data
#         self.info_gain, self.feature_num, self.split_num = 0, 0, 0
        
    def info_gain(self, row, col):
        pos, neg = func_count_one(self.flag_data)
        init_entropy = func_calc_entropy(pos, neg)
        row_count, col_count = self.norm_data.shape        
        upper_pos_count, upper_neg_count = 0, 0
        lower_pos_count, lower_neg_count = 0, 0        
        for j, value in enumerate(self.norm_data.T[row]):
            if value > col:
                if self.flag_data[j] > 0: upper_pos_count += 1
                else: upper_neg_count += 1
            else:
                if self.self.flag_data[j] > 0: lower_pos_count += 1
                else: lower_neg_count += 1        
        upper_entropy = func_calc_entropy(upper_pos_count, upper_neg_count)
        upper_p = (float(upper_pos_count + upper_neg_count) / row_count) * upper_entropy
        lower_entropy = func_calc_entropy(lower_pos_count, lower_neg_count)
        lower_p = ((float)(lower_pos_count + upper_neg_count) / row_count) * lower_entropy 
        return init_entropy - upper_p - lower_p

def func_count_one(flag_data):
    pos, neg = 0, 0
    for data in flag_data:
        if data == 1: pos += 1
        if data == -1: neg += 1
    return pos, neg
        
def func_calc_entropy(pos, neg):
    lhs_prob, rhs_prob = float(pos) / (pos + neg), float(neg) / (pos + neg)
    entropy = lhs_prob * np.log(lhs_prob) + rhs_prob * np.log(rhs_prob)
    return -entropy

In [157]:
def func_plot_data(fig, x_list, y_list, label, title='Training-Testing Data'):
    plt.plot(x_list, y_list, label=label)
    plt.legend()
    fig.suptitle(title + ' Error', fontsize=16)
    plt.xlabel('K-Value', fontsize=16)
    plt.ylabel('Error', fontsize=16)

def func_part_one():
    train_data, test_data = DataFormat('knn_train.csv'), DataFormat('knn_test.csv')
    train_norm_data, train_flag_data = train_data.func_norm_data()
    test_norm_data, test_flag_data = test_data.func_norm_data()
    knn_train, knn_test = KNN(train_norm_data, train_flag_data), KNN(test_norm_data, test_flag_data)
    k_list, train_error_list, cross_valid_list, test_error_list = range(1, 52, 2), [], [], []
    
    for k in k_list:
        train_error = func_calc_error(train_data, knn_train, k)
        train_error_list.append(train_error)
        cross_valid = func_cross_valid_error(train_data, knn_train, k)
        cross_valid_list.append(cross_valid)
        test_error = func_calc_error(test_data, knn_test, k)
        test_error_list.append(test_error)

    fig = plt.figure(figsize=(9,6))
    print 'Training:', train_error_list
    func_plot_data(fig, k_list, train_error_list, 'Training Data')
    print 'Testing:', test_error_list    
    func_plot_data(fig, k_list, test_error_list, 'Testing Data')
    fig = plt.figure(figsize=(9,6))
    print 'Cross Valid:', cross_valid_list
    func_plot_data(fig, k_list, cross_valid_list, 'Cross Validation', 'Cross Validation')

def func_part_two():
    train_data = DataFormat('knn_train.csv')
    train_norm_data, train_flag_data = train_data.func_norm_data()
    pos, neg = func_count_one(train_flag_data)
    print pos, neg, func_calc_entropy(pos, neg)
    stump = DecisionStump(train_norm_data, train_flag_data)
    stump.info_gain()
    
if __name__ == '__main__':
#     func_part_one()
    func_part_two()

101 183 0.650864596714
284 30
